# Cartographies creation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from sklearn import svm
from matplotlib.widgets import Button
import ipywidgets as widgets

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from scipy.io.wavfile import write

#Direct input 
plt.rcParams['text.latex.preamble']= r"\usepackage{lmodern}"
#Options
params = {'text.usetex' : True,
          'font.size' : 13,
          'font.family' : 'lmodern',
          }
plt.rcParams.update(params) 

%matplotlib qt5

In [2]:
from cProfile import label


class interactivePlotClassificationBase:
    def __init__(self, X, Y, labels, params) -> None:
        self.X = X
        self.Y = Y
        self.i = 0
        self.labels = labels
        self.params=params

        self.t1 = self.X[int(1/3*len(self.X))]
        self.ind_t1 = int(1/3*len(self.X))

        self.fig = plt.figure()
        self.ax = plt.axes()
        self.plot()

        self.button1 = widgets.Button(description="Previous")
        self.button2 = widgets.Button(description="Next")
        out = widgets.Output()
        
        buttons = widgets.VBox(children=[self.button1, self.button2])
        all_widgets = widgets.HBox(children=[buttons, out])
        display(all_widgets)

        self.button1.on_click(self.callback_prev)
        self.button2.on_click(self.callback_next)
    
    def plot(self):
        self.ax = plt.axes()
        self.ax.plot(self.X, self.Y[self.i], label=f"P_0={self.params[self.i, 0]:.2f}, h_r={self.params[self.i, 1]:.5f}")
        self.ax.vlines(self.t1, ymin=[np.min(self.Y[0])], ymax=[np.max(self.Y[0])], label=r"$t_1$", color='r', linestyle = '--')
        self.ax.hlines([np.sqrt(np.mean(self.Y[self.i, :self.ind_t1]**2)), np.sqrt(np.mean(self.Y[self.i, self.ind_t1:]**2))], xmin=[0, self.t1], xmax=[self.t1, self.X[-1]], label=r"$\bar{P}$", color = 'orange',linestyle = '--')
        self.ax.text(0.6*np.max(self.X), 0.6*np.max(self.Y[self.i]), f"label = {self.labels[self.i]}")
        
        scaled = np.int16(self.Y[self.i]/np.max(np.abs(self.Y[self.i])) * 32767)
        write('test.wav', 20000, scaled)
        plt.legend()
        
    def prev(self):
        if self.i==0:
            self.i=len(self.Y[:, 0])-1
        else:
            self.i-=1

    def next(self):
        if self.i==len(self.Y[:, 0])-1:
            self.i=0
        else:
            self.i+=1
    
    def callback_prev(self, widget):
        self.ax.cla()
        self.prev()
        self.plot()

    def callback_next(self, widget):
        self.fig.clf()
        self.next()
        self.plot()

class interactivePlotClassificationMoy(interactivePlotClassificationBase):
    def __init__(self, X, Y, Ymoy, labels) -> None:
        self.Ymoy = Ymoy
        super().__init__(X, Y, labels)
        

    def plot(self):
        self.ax = plt.axes()
        self.ax.plot(self.X, self.Y[self.i])
        self.ax.plot(self.X[:len(self.Ymoy[self.i])], self.Ymoy[self.i], color = 'green')
        self.ax.vlines(self.t1, ymin=[np.min(self.Y[0])], ymax=[np.max(self.Y[0])], label=r"$t_1$", color='r', linestyle = '--')
        self.ax.text(0.6*np.max(self.X), 0.6*np.max(self.Y[self.i]), f"label = {self.labels[self.i]}")
        plt.legend()
        


In [106]:
# Reading a file
filename  ='./results/P0hra0/a0_0.hdf5'

fixedParams = {}
variableParams = {}
results = {}
with h5py.File(filename, "r") as f:
    # Fixed parameters
    for key in f["model/fixed_parameters"].attrs.keys():
        fixedParams[key] = f["model/fixed_parameters"].attrs[key]
    # Variable parameters
    for key, item in f["model/variable_parameters"].items():
        variableParams[key] = item[()]
    N_sim = len(variableParams[key])
    timevect = f["simulation0"]["time"][()]
    results["time"] = timevect
    results["N_sim"] = N_sim

    for i in range(N_sim):
        sim_results = f[f"simulation{i}"]
        results[f'simulation{i}'] = {
            "efforts" : sim_results["efforts"][()].reshape(len(timevect), 19).T,
            "flows" : sim_results["flows"][()].reshape(len(timevect), 19).T,
            "states" : sim_results["states"][()].reshape(len(timevect)+1, 12).T,
            "P0": f["model/variable_parameters"]["P0"][()][i],
            "hr": f["model/variable_parameters"]["hr"][()][i],
            "a0": f["model/variable_parameters"]["a0"][()][i],
            "success": sim_results.attrs["success"]
            }

In [110]:
def oscillationsBase(results, plot=True):
    alpha = 0.9
    time = results["time"]
    t1 = int(len(time)/3)

    Y = np.zeros(results["N_sim"])

    N_features = 3

    X= np.zeros((results["N_sim"], N_features))

    Ps = np.zeros((N_sim, len(time)))
    Qs = np.zeros((N_sim, len(time)))
    hs = np.zeros((N_sim, len(time)))

    for i in range(results["N_sim"]):
        X[i, 0] = results[f"simulation{i}"]["P0"]
        X[i, 1] = results[f"simulation{i}"]["hr"]
        X[i, 2] = results[f"simulation{i}"]["a0"]

        P = results[f"simulation{i}"]["states"][-2, :-1]*results[f"simulation{i}"]["a0"]
        Ps[i, :] = P
        pixeff = results[f"simulation{i}"]["efforts"][6, :]
        piexpeff = results[f"simulation{i}"]["efforts"][8, :]
        Qs[i, :] = fixedParams['L0']*fixedParams['hr'] * pixeff-2*fixedParams['L0']*fixedParams['l0']*piexpeff
        hs[i] = results[f"simulation{i}"]["states"][9, :-1]
        Pmax = np.sqrt(np.mean(P[:t1]**2))
        if results[f"simulation{i}"]["success"]:
            if np.sqrt(np.mean(P[t1:]**2)) > alpha*Pmax:
                results[f"simulation{i}"]["oscillations"] = True
                Y[i]= 1
            else:
                results[f"simulation{i}"]["oscillations"] = False
                Y[i]= 0
        else:
            results[f"simulation{i}"]["oscillations"] = False
            Y[i]= -1

    if plot:
        try:
            interactivePlotClassificationBase(time, Ps[Y==1, :], Y[Y==1], X)
        except:
            print('No oscillations...')
            return X, Y

    return X, Y
        
X, Y = oscillationsBase(results, plot=True)

In [113]:
# Figure
fig = plt.figure(figsize=(6, 4))
N_features = 3
if N_features==2:
    ax = fig.add_subplot()#projection='3d')

    ax.scatter(X[Y==0, 0], X[Y==0, 1], label="Pas d'oscillations")
    ax.scatter(X[Y==1, 0], X[Y==1, 1], label='Oscillations', color='r')
    ax.scatter(X[Y==-1, 0], X[Y==-1, 1], label='Non convergence : choc des plis vocaux', color='black')
    plt.xlabel(r'$P_0$ [Pa]')
    plt.ylabel(r'$h_r$ [m]')
else:
    ax = fig.add_subplot(projection='3d')

    ax.scatter(X[Y==0, 0], X[Y==0, 1], X[Y==0, 2], label="Pas d'oscillations")
    ax.scatter(X[Y==1, 0], X[Y==1, 1], X[Y==1, 2], label='Oscillations', color='r')
    ax.scatter(X[Y==-1, 0], X[Y==-1, 1], X[Y==-1, 2], label='Non convergence : choc des plis vocaux', color='black')
    plt.xlabel(r'$P_0$ [Pa]')
    plt.ylabel(r'$h_r$ [m]')    
    plt.gca().set_zlabel(r'$a_0$ [kg.m$^-2$]') 

plt.gca().legend(loc='upper center', bbox_to_anchor=(0.5, +1.15),
          fancybox=True, shadow=True, ncol=3)

In [100]:
clf = make_pipeline(StandardScaler(), svm.SVC(C=1, kernel='rbf',gamma=3, decision_function_shape='ovo'))
clf.fit(X, Y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=1, decision_function_shape='ovo', gamma=3))])

In [101]:
def make_meshgrid(x, y, hx=.02, hy=0.02):
    x_min, x_max = x.min() - hx, x.max() + hx
    y_min, y_max = y.min() - hy, y.max() + hy
    xx, yy = np.meshgrid(np.arange(x_min, x_max, hx), np.arange(y_min, y_max, hy))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    #Z = (-1 * Z[:, 0] + 1 * Z[:, 2]) / np.sum(Z, axis=1)
    Z = Z.reshape(xx.shape)
    out = ax.contour(xx, yy, Z, vmin=-0.5, vmax=0.5, **params)
    return out

fig, ax = plt.subplots()
# title for the plots
title = ('SVC classification')
# Set-up grid for plotting.
X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1, hx=1, hy = 1e-6)

cs = plot_contours(ax, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)




ax.scatter(X[Y==0, 0], X[Y==0, 1], label="Pas d'oscillations")
ax.scatter(X[Y==1, 0], X[Y==1, 1], label='Oscillations', color='r')
ax.scatter(X[Y==-1, 0], X[Y==-1, 1], label='Non convergence : choc des plis vocaux', color='black')
ax.set_ylabel(r'$h_r$ [m]')
ax.set_xlabel(r'$P_0$ [Pa]')
#ax.set_xticks(())
#ax.set_yticks(())
plt.gca().legend(loc='upper center', bbox_to_anchor=(0.5, +1.15),
          fancybox=True, shadow=True, ncol=3)
plt.show()

In [ ]:
# Another idea of oscillation detection function
def oscillationsMoy(results, plot=True):
    alpha = 0.9
    time = results["time"]
    t1 = int(len(time)/3)
    t2 = int(2*len(time)/3)

    Y = np.zeros(results["N_sim"])

    N_features = 3
    N_moy = 1000

    X= np.zeros((results["N_sim"], N_features))

    Ps = np.zeros((N_sim, len(time)))
    Psmoy = np.zeros((N_sim, len(time)-N_moy+1))

    for i in range(results["N_sim"]):
        X[i, 0] = results[f"simulation{i}"]["P0"]
        X[i, 1] = results[f"simulation{i}"]["a0"]
        X[i, 2] = results[f"simulation{i}"]["r"]

        P = results[f"simulation{i}"]["states"][-2, :-1]
        Ps[i, :] = P
        Psmoy[i, :] = np.convolve(np.abs(P), np.ones(N_moy), 'valid') / N_moy
        Pmax = np.sqrt(np.mean(P[:t1]**2))
        if results[f"simulation{i}"]["success"]:
            if Psmoy[i, t2] > alpha*Psmoy[i, t1]:
                results[f"simulation{i}"]["oscillations"] = True
                Y[i]= 1
            else:
                results[f"simulation{i}"]["oscillations"] = False
                Y[i]= 0
        else:
            results[f"simulation{i}"]["oscillations"] = False
            Y[i]= -1

    if plot:
        interactivePlotClassificationMoy(time, Ps[Y==1, :], Psmoy[Y==1, :], Y[Y==1])

    return X, Y
        
X, Y = oscillationsMoy(results)